In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import seaborn as sns
import re
import random as rd
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from nltk import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords, wordnet
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import autokeras as ak
from sklearn.model_selection import KFold

import nltk
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ddayv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ddayv\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
def Tokenize(f):     ## Pre-processando a frase

    ## Colocando em minusculo
    ## Retirando a pontuaçao
    ## Retirando as StopWords
    
    f = f.lower().replace('\n', '').replace('-','').replace('#','').replace('.','').replace(',','').replace('!','').replace('r\n','').replace('  ','').replace('https','').replace('rt','').replace('rn','')
    token = RegexpTokenizer(r"\w+")
    f = token.tokenize(f)
    
    stop_words = set(stopwords.words('portuguese'))
    
    new_word = [word for word in f if not word in stop_words]
    
    return ' '.join(new_word)

def remove_user(frase):
    frase = re.sub('@\w+','',frase)
    frase = re.sub('{https}[^ ]+','',frase)
    frase = re.sub('https\w+','',frase)
    # re.sub('#\w+','',frase)
    return frase

def pre_X(frases):
    lista = []
    
    for frase in frases:
        lista.append(frase)
        
    return lista

def pre_Y(number):
    lista = []
    
    for numb in number:
        lista.append(numb)
    
    return lista

def set_array(frases):
    
    vocab = []
    palavras = []
    for frase in frases:
        
        text_array = remove_user(frase)
        text_array = Tokenize(text_array)
        text_array = text_array.split(' ')
        for i in range(len(text_array)):
            vocab.append(text_array[i])
        
    return vocab


## TWEETSENT

In [3]:
df_tweet= pd.read_csv('export_TweetSentBR.csv')
df_tweet

,id,id_twitter,text,sentiment
0,1343,863044774588272640,Que coisa linda! O Programa #encontro estava m...,1
1,1344,865583716088766467,"Por mais #Encontro com as Irmãs Galvão, adorei...",1
2,1345,865063232201011201,Mr. CATRA @OficialMrCatra lançando sua nova mú...,1
3,1346,864668391008763905,quem viu aquela lutadora modela barbuda tatuad...,0
4,1347,865572794016378882,Tô passada com esse cara.... quanta merda pode...,-1
...,...,...,...,...
11565,12908,864636619000877056,eu ja to aqui pronto pro #MasterChefBR mas ain...,-1
11566,12909,863581588713603072,MALUCO! Uma coisa que eu não tenho coragem é e...,-1
11567,12910,864831041349054464,#MaisVoce @ANAMARIABRAGA está linda @RedeGlobo,1
11568,12911,863042798575951872,"Que orgulho de ti, @sportrecife! #Encontro",1


In [4]:
df_remove = df_tweet[df_tweet['sentiment'] == '-']
df_tweet = df_tweet.drop(df_remove.index)

df_tweet['text'] = df_tweet['text'].apply(remove_user)
df_tweet['text'] = df_tweet['text'].apply(Tokenize)

In [5]:
df_tweet['sentiment'] = df_tweet['sentiment'].apply(lambda x: int(x))

In [6]:
text_Tweetsent = df_tweet['text']
polarity_ = np.asarray(df_tweet['sentiment'])

In [7]:
count_vect = CountVectorizer()
X_text = count_vect.fit_transform(text_Tweetsent)
tfidf_transformer = TfidfTransformer()
X_train_TWEET = tfidf_transformer.fit_transform(X_text) # Aplicando o TF-IDF
X_train_TWEET.shape

(11533, 15334)

## TASH

In [8]:
df1 = pd.read_csv("tash-pt.csv")
df1 = df1.dropna()
df1 = df1.drop(columns=['id_twitter'])


In [9]:
df1['text'] = df1['text'].apply(remove_user)
df1['text'] = df1['text'].apply(Tokenize)

In [10]:
text_TASH = df1['text']
sentiment_ = np.asarray(df1['sentiment'])

X_text_TASH = count_vect.transform(text_TASH)
X_text_TASH_ = tfidf_transformer.transform(X_text_TASH) # Aplicando o TF-IDF
X_text_TASH_.shape

(2787, 15334)

## KANSOAN

In [11]:
df = pd.read_csv("data.txt", sep=',', header=None, names=['text','sentiment'])

df_remove = df[df['sentiment'] == '#Inveja'].index
df = df.drop(df_remove)

df_remove = df[df['sentiment'] == '#Raiva'].index
df = df.drop(df_remove)

df_remove = df[df['sentiment'] == '#Ironia'].index
df = df.drop(df_remove)


def binario(termo):
    if termo == '#Feliz' or termo == '#Amor':
        return 1
    elif termo == '#Triste' or termo == '#Chateado':
        return 0       
    
df['sentiment'] = df['sentiment'].apply(binario)

In [12]:
df['text'] = df['text'].apply(remove_user)
df['text'] = df['text'].apply(Tokenize)

In [13]:
text_KANSOAN = df['text']
sentiment = np.asarray(df['sentiment'])

X_text_KANSOAN = count_vect.transform(text_KANSOAN)
X_text_KANSOAN_ = tfidf_transformer.transform(X_text_KANSOAN) # Aplicando o TF-IDF
X_text_KANSOAN_.shape

(9224, 15334)

## UNILEX

In [14]:
df1 = pd.read_excel('./rotulação/TweetsPolitical01 OK OK.xlsx')
df2 = pd.read_excel('./rotulação/TweetsPolitical02 OK OK.xlsx')
df3 = pd.read_excel('./rotulação/TweetsPolitical03 OK OK.xlsx')
df4 = pd.read_excel('./rotulação/TweetsPolitical04 OK OK.xlsx')
df5 = pd.read_excel('./rotulação/TweetsPolitical05 OK OK.xlsx')
df6 = pd.read_excel('./rotulação/TweetsPolitical06 OK OK.xlsx')
df7 = pd.read_excel('./rotulação/TweetsPolitical07 OK OK.xlsx')
df8 = pd.read_excel('./rotulação/TweetsPolitical08 OK OK.xlsx')
df9 = pd.read_excel('./rotulação/TweetsPolitical09 OK OK.xlsx')
df10 = pd.read_excel('./rotulação/TweetsPolitical10 OK OK.xlsx')
df11 = pd.read_excel('./rotulação/TweetsPolitical11 OK OK.xlsx')
df12 = pd.read_excel('./rotulação/TweetsPolitical12 OK OK.xlsx')
df13 = pd.read_excel('./rotulação/TweetsPolitical13 OK OK.xlsx')
df14 = pd.read_excel('./rotulação/TweetsPolitical14 OK OK.xlsx')
df15 = pd.read_excel('./rotulação/TweetsPolitical15 OK OK.xlsx')

lista = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15]

df_all = pd.concat(lista)
df_all.shape

(12670, 2)

In [15]:
df_all = df_all.dropna()

In [16]:
df_all.drop(df_all[df_all['Polaridade'] == 11].index, inplace=True)
df_all.drop(df_all[df_all['Polaridade'] == -2].index, inplace=True)
df_all.drop(df_all[df_all['Polaridade'] == 10].index, inplace=True)

df_all['Polaridade'].unique()

array([ 0.,  1., -1.])

In [17]:
df_all.head()

,Tweet,Polaridade
0,- #CaoNossoDeCadaDia #Novo - Vanessa Mandotti ...,0.0
1,"- Bola pra frente que amanhã é outro dia, outr...",1.0
2,- Cara de mal? Acho que não... Apenas um corte...,1.0
3,""" #PCdoB O PARTIDO QUE FAZ A DIFERENÇA: LEAL E...",1.0
4,"""@camaradamae: #PCdoB o Partido da coragem ! ...",1.0


In [18]:
df_all['Tweet'] = df_all['Tweet'].apply(remove_user)
df_all['Tweet'] = df_all['Tweet'].apply(Tokenize)
polarity = np.asarray(df_all['Polaridade'])
Tweet = df_all['Tweet']


In [19]:
Tweet = count_vect.transform(Tweet)
X_text_UNILEX = tfidf_transformer.transform(Tweet) # Aplicando o TF-IDF
X_text_UNILEX.shape

(12626, 15334)

## NB - TRAIN -> TWEETSENT- TEST -> UNILEX

In [20]:
## TIRAR OS SPLITS , até no treino

In [21]:
clf = MultinomialNB()
clf.fit(X_train_TWEET, polarity_)
y_pred = clf.predict(X_text_UNILEX)

print(classification_report(polarity,y_pred))

              precision    recall  f1-score   support

        -1.0       0.50      0.44      0.47      4186
         0.0       0.37      0.22      0.28      4740
         1.0       0.35      0.58      0.44      3700

    accuracy                           0.40     12626
   macro avg       0.41      0.41      0.39     12626
weighted avg       0.41      0.40      0.39     12626



## NB - TRAIN -> TWEETSENT - TEST -> TASH

In [22]:
clf = MultinomialNB()
clf.fit(X_train_TWEET, polarity_)
y_pred = clf.predict(X_text_TASH_)

print(classification_report(sentiment_,y_pred))

              precision    recall  f1-score   support

          -1       0.45      0.48      0.46       881
           0       0.46      0.08      0.13      1018
           1       0.39      0.73      0.51       888

    accuracy                           0.41      2787
   macro avg       0.43      0.43      0.37      2787
weighted avg       0.43      0.41      0.35      2787



## NB - TRAIN -> TWEET - TEST -> KANSOAN

In [23]:
def func(num):
    if num == 0:
        return -1
    else:
        return 1

sentiment = df['sentiment'].apply(func)


In [24]:
clf = MultinomialNB()
clf.fit(X_train_TWEET, polarity_)
y_pred = clf.predict(X_text_KANSOAN_)

print(classification_report(sentiment,y_pred,zero_division=True))

              precision    recall  f1-score   support

          -1       0.94      0.76      0.84      4504
           0       0.00      1.00      0.00         0
           1       0.82      0.94      0.87      4720

    accuracy                           0.85      9224
   macro avg       0.59      0.90      0.57      9224
weighted avg       0.88      0.85      0.86      9224



## SVM - TRAIN->TWEETSENT - TEST-> UNILEX

In [25]:
clf = SVC().fit(X_train_TWEET, polarity_) 
y_pred = clf.predict(X_text_UNILEX)

print(classification_report(polarity,y_pred))

              precision    recall  f1-score   support

        -1.0       0.52      0.37      0.43      4186
         0.0       0.37      0.25      0.30      4740
         1.0       0.34      0.59      0.43      3700

    accuracy                           0.39     12626
   macro avg       0.41      0.40      0.39     12626
weighted avg       0.41      0.39      0.38     12626



## SVM T TWEET T TASH

In [26]:
clf = SVC().fit(X_train_TWEET, polarity_) 
y_pred = clf.predict(X_text_TASH_)

print(classification_report(sentiment_,y_pred))

              precision    recall  f1-score   support

          -1       0.43      0.50      0.46       881
           0       0.43      0.12      0.18      1018
           1       0.40      0.66      0.50       888

    accuracy                           0.41      2787
   macro avg       0.42      0.43      0.38      2787
weighted avg       0.42      0.41      0.37      2787



## SVM T TWEET T KANSOAN

In [27]:
clf = SVC().fit(X_train_TWEET, polarity_) 
y_pred = clf.predict(X_text_KANSOAN_)

print(classification_report(sentiment,y_pred, zero_division=True))

              precision    recall  f1-score   support

          -1       0.93      0.74      0.83      4504
           0       0.00      1.00      0.00         0
           1       0.82      0.91      0.86      4720

    accuracy                           0.83      9224
   macro avg       0.58      0.88      0.56      9224
weighted avg       0.87      0.83      0.85      9224



## RL T TWEETSENT T UNILEX

In [28]:
clf = LogisticRegression(max_iter=1000).fit(X_train_TWEET, polarity_) 
y_pred = clf.predict(X_text_UNILEX)

print(classification_report(polarity,y_pred))

              precision    recall  f1-score   support

        -1.0       0.49      0.44      0.47      4186
         0.0       0.40      0.37      0.38      4740
         1.0       0.36      0.44      0.40      3700

    accuracy                           0.41     12626
   macro avg       0.42      0.42      0.41     12626
weighted avg       0.42      0.41      0.41     12626



## RL T TWEET T TASH

In [29]:
clf = LogisticRegression(max_iter=1000).fit(X_train_TWEET, polarity_) 
y_pred = clf.predict(X_text_TASH_)

print(classification_report(sentiment_,y_pred))

              precision    recall  f1-score   support

          -1       0.42      0.57      0.48       881
           0       0.44      0.19      0.27      1018
           1       0.43      0.56      0.49       888

    accuracy                           0.43      2787
   macro avg       0.43      0.44      0.41      2787
weighted avg       0.43      0.43      0.41      2787



## RL T TWEET T KANSOAN

In [30]:
clf = LogisticRegression(max_iter=1000).fit(X_train_TWEET, polarity_) 
y_pred = clf.predict(X_text_KANSOAN_)

print(classification_report(sentiment,y_pred, zero_division=True))

              precision    recall  f1-score   support

          -1       0.92      0.80      0.86      4504
           0       0.00      1.00      0.00         0
           1       0.88      0.88      0.88      4720

    accuracy                           0.84      9224
   macro avg       0.60      0.90      0.58      9224
weighted avg       0.90      0.84      0.87      9224



In [31]:
one = OneHotEncoder(sparse=False)
teste_ = one.fit_transform(np.asarray(sentiment).reshape(-1,1))
nome = pd.DataFrame(teste_)
nome['coluna'] = 0
nome.rename(columns={1: 2, 'coluna':1} , inplace=True)
Y_KANSOAN = nome[[0,1,2]].to_numpy()

## TOTALMENT.C T TWEETSENT T UNILEX

In [32]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(25, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(10, activation='tanh'),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(3 , activation='softmax')
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

y_one = one.fit_transform(np.asarray(polarity_).reshape(-1,1))
y_one_test = one.fit_transform(polarity.reshape(-1,1))

fit = model.fit(X_train_TWEET.todense(), y_one, epochs=5, validation_data=(X_text_UNILEX.todense(), y_one_test))

Epoch 1/5
361/361 [==============================] - 3s 9ms/step - loss: 0.9921 - accuracy: 0.5082 - val_loss: 1.0672 - val_accuracy: 0.4118
Epoch 2/5
361/361 [==============================] - 2s 7ms/step - loss: 0.7055 - accuracy: 0.7100 - val_loss: 1.1106 - val_accuracy: 0.4537
Epoch 3/5
361/361 [==============================] - 2s 7ms/step - loss: 0.4545 - accuracy: 0.8427 - val_loss: 1.4226 - val_accuracy: 0.4400
Epoch 4/5
361/361 [==============================] - 3s 7ms/step - loss: 0.2893 - accuracy: 0.9062 - val_loss: 1.7926 - val_accuracy: 0.4444
Epoch 5/5
361/361 [==============================] - 2s 6ms/step - loss: 0.1966 - accuracy: 0.9415 - val_loss: 2.0648 - val_accuracy: 0.4435


In [33]:
predicted = model.predict(X_text_UNILEX.todense())

print(classification_report(np.argmax(y_one_test, axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.47      0.50      0.48      4186
           1       0.44      0.49      0.46      4740
           2       0.42      0.33      0.37      3700

    accuracy                           0.44     12626
   macro avg       0.44      0.44      0.44     12626
weighted avg       0.44      0.44      0.44     12626



## TASH

In [34]:
predicted = model.predict(X_text_TASH_.todense())

print(classification_report(np.argmax(one.fit_transform(sentiment_.reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.40      0.55      0.47       881
           1       0.42      0.29      0.34      1018
           2       0.43      0.43      0.43       888

    accuracy                           0.42      2787
   macro avg       0.42      0.42      0.41      2787
weighted avg       0.42      0.42      0.41      2787



## KANSOAN

In [35]:
predicted = model.predict(X_text_KANSOAN_.todense())

print(classification_report(np.argmax(Y_KANSOAN, axis=1), np.argmax(predicted, axis=1), zero_division=True))

              precision    recall  f1-score   support

           0       0.85      0.78      0.81      4504
           1       0.00      1.00      0.00         0
           2       0.91      0.69      0.78      4720

    accuracy                           0.73      9224
   macro avg       0.59      0.82      0.53      9224
weighted avg       0.88      0.73      0.80      9224



## LSTM T TWEET T TASH

In [36]:
vectorize_layer = TextVectorization(
 max_tokens=15000,
 output_mode='int',
 output_sequence_length=len(max(df_tweet['text'] )))

vocab = set_array(df_tweet['text'] )

vectorize_layer.adapt(np.unique(vocab))
len(vectorize_layer.get_vocabulary())


15000

In [37]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=64,mask_zero=True),
    
    tf.keras.layers.LSTM(50, activation='relu' ,return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.LSTM(25 , activation='tanh', return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.LSTM(10 , activation='tanh'),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(3, activation='softmax')
])


model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

y_one = one.fit_transform(np.asarray(polarity_).reshape(-1,1))
y_one_test = one.fit_transform(np.asarray(sentiment_).reshape(-1,1))

fit = model.fit(np.asarray(pre_X(text_Tweetsent)), y_one, epochs=5, batch_size=128 ,validation_data=(np.asarray(pre_X(text_TASH)), y_one_test))

Epoch 1/5
91/91 [==============================] - 15s 169ms/step - loss: 1.0446 - accuracy: 0.4713 - val_loss: 1.1898 - val_accuracy: 0.3394
Epoch 2/5
91/91 [==============================] - 15s 163ms/step - loss: 0.8119 - accuracy: 0.6403 - val_loss: 1.1718 - val_accuracy: 0.4177
Epoch 3/5
91/91 [==============================] - 15s 165ms/step - loss: 0.5820 - accuracy: 0.7797 - val_loss: 1.3728 - val_accuracy: 0.4406
Epoch 4/5
91/91 [==============================] - 15s 166ms/step - loss: 0.4179 - accuracy: 0.8573 - val_loss: 1.6534 - val_accuracy: 0.4334
Epoch 5/5
91/91 [==============================] - 15s 165ms/step - loss: 0.3197 - accuracy: 0.8959 - val_loss: 1.8895 - val_accuracy: 0.4327


In [38]:
predicted = model.predict(np.asarray(pre_X(text_TASH)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(sentiment_).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.42      0.46      0.44       881
           1       0.42      0.59      0.49      1018
           2       0.49      0.23      0.31       888

    accuracy                           0.43      2787
   macro avg       0.45      0.43      0.41      2787
weighted avg       0.44      0.43      0.42      2787



## UNILEX

In [39]:
predicted = model.predict(np.asarray(pre_X(df_all['Tweet'])))

print(classification_report(np.argmax(one.fit_transform(np.asarray(polarity).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.52      0.29      0.37      4186
           1       0.42      0.83      0.56      4740
           2       0.55      0.13      0.21      3700

    accuracy                           0.45     12626
   macro avg       0.49      0.42      0.38     12626
weighted avg       0.49      0.45      0.39     12626



## KANSOAN

In [40]:
predicted = model.predict(np.asarray(pre_X(text_KANSOAN)))

print(classification_report(np.argmax(Y_KANSOAN, axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.80      0.59      0.68      4504
           1       0.00      0.00      0.00         0
           2       0.87      0.32      0.47      4720

    accuracy                           0.45      9224
   macro avg       0.56      0.30      0.38      9224
weighted avg       0.84      0.45      0.57      9224



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## CONV1D T TWEET T KANSOAN

In [41]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=64,
        mask_zero=True),
    
    tf.keras.layers.Conv1D(32,6, activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')   
])

model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

y_one = one.fit_transform(np.asarray(polarity_).reshape(-1,1))
y_one_test = Y_KANSOAN

fit = model.fit(np.asarray(pre_X(text_Tweetsent)), y_one, epochs=5, batch_size=128, validation_data=(np.asarray(pre_X(text_KANSOAN)),Y_KANSOAN))

Epoch 1/5
91/91 [==============================] - 3s 32ms/step - loss: 1.0705 - accuracy: 0.4508 - val_loss: 1.0051 - val_accuracy: 0.5117
Epoch 2/5
91/91 [==============================] - 3s 33ms/step - loss: 0.9428 - accuracy: 0.5223 - val_loss: 1.3561 - val_accuracy: 0.4339
Epoch 3/5
91/91 [==============================] - 3s 36ms/step - loss: 0.6964 - accuracy: 0.6647 - val_loss: 1.2540 - val_accuracy: 0.5730
Epoch 4/5
91/91 [==============================] - 3s 35ms/step - loss: 0.5472 - accuracy: 0.7356 - val_loss: 1.6655 - val_accuracy: 0.3987
Epoch 5/5
91/91 [==============================] - 3s 36ms/step - loss: 0.4366 - accuracy: 0.8110 - val_loss: 2.2997 - val_accuracy: 0.3426


In [42]:
predicted = model.predict(np.asarray(pre_X(text_KANSOAN)))

print(classification_report(np.argmax(Y_KANSOAN, axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.61      0.37      0.46      4504
           1       0.00      0.00      0.00         0
           2       0.88      0.32      0.47      4720

    accuracy                           0.34      9224
   macro avg       0.50      0.23      0.31      9224
weighted avg       0.75      0.34      0.46      9224



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## UNILEX

In [43]:
predicted = model.predict(np.asarray(pre_X(df_all['Tweet'])))

print(classification_report(np.argmax(one.fit_transform(np.asarray(polarity).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.38      0.20      0.26      4186
           1       0.41      0.85      0.55      4740
           2       0.56      0.09      0.15      3700

    accuracy                           0.41     12626
   macro avg       0.45      0.38      0.32     12626
weighted avg       0.44      0.41      0.34     12626



## TASH

In [44]:
predicted = model.predict(np.asarray(pre_X(text_TASH)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(sentiment_).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.34      0.44      0.38       881
           1       0.41      0.51      0.45      1018
           2       0.49      0.21      0.29       888

    accuracy                           0.39      2787
   macro avg       0.41      0.38      0.37      2787
weighted avg       0.41      0.39      0.38      2787



## BDR T TWEET T UNILEX

In [46]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=64,
        mask_zero=True),
    
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
    
])

model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

y_one = one.fit_transform(np.asarray(polarity_).reshape(-1,1))
y_one_test = one.fit_transform(np.asarray(polarity).reshape(-1,1))

fit = model.fit(np.asarray(pre_X(text_Tweetsent)), y_one, epochs=5, batch_size=128 ,validation_data=(np.asarray(pre_X(df_all['Tweet'])), y_one_test))

Epoch 1/5
91/91 [==============================] - 40s 444ms/step - loss: 0.9747 - accuracy: 0.5248 - val_loss: 1.0515 - val_accuracy: 0.4659
Epoch 2/5
91/91 [==============================] - 43s 472ms/step - loss: 0.6793 - accuracy: 0.7144 - val_loss: 1.6087 - val_accuracy: 0.4077
Epoch 3/5
91/91 [==============================] - 42s 462ms/step - loss: 0.4258 - accuracy: 0.8448 - val_loss: 1.6513 - val_accuracy: 0.4504
Epoch 4/5
91/91 [==============================] - 43s 468ms/step - loss: 0.2636 - accuracy: 0.9102 - val_loss: 2.0044 - val_accuracy: 0.4676
Epoch 5/5
91/91 [==============================] - 43s 472ms/step - loss: 0.1861 - accuracy: 0.9379 - val_loss: 2.5586 - val_accuracy: 0.4526


In [47]:
predicted = model.predict(np.asarray(pre_X(df_all['Tweet'])))

print(classification_report(np.argmax(one.fit_transform(np.asarray(polarity).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.50      0.39      0.44      4186
           1       0.43      0.79      0.55      4740
           2       0.56      0.09      0.15      3700

    accuracy                           0.45     12626
   macro avg       0.50      0.42      0.38     12626
weighted avg       0.49      0.45      0.40     12626



## TASH

In [48]:
predicted = model.predict(np.asarray(pre_X(text_TASH)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(sentiment_).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.41      0.47      0.44       881
           1       0.40      0.58      0.47      1018
           2       0.49      0.17      0.25       888

    accuracy                           0.42      2787
   macro avg       0.43      0.41      0.39      2787
weighted avg       0.43      0.42      0.39      2787



## KANSOAN

In [49]:
predicted = model.predict(np.asarray(pre_X(text_KANSOAN)))

print(classification_report(np.argmax(Y_KANSOAN, axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.88      0.75      0.81      4504
           1       0.00      0.00      0.00         0
           2       0.93      0.30      0.46      4720

    accuracy                           0.52      9224
   macro avg       0.60      0.35      0.42      9224
weighted avg       0.90      0.52      0.63      9224



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
